Documentation for the Taxonomic Information Registry is lagging way behind. I've been dumping properties of various kinds into the TIR table's key/value pairs with no documentation anywhere. I built this notebook to help document the properties by building a property registry table containing property names, types, definitions, and maybe some additional information in future.

In [15]:
import requests,configparser


In [16]:
# Get API keys and any other config details from a file that is external to the code.
config = configparser.RawConfigParser()
config.read_file(open(r'../config/stuff.py'))

In [17]:
# Build base URL with API key using input from the external config.
def getBaseURL():
    gc2APIKey = config.get('apiKeys','apiKey_GC2_BCB').replace('"','')
    apiBaseURL = "https://gc2.mapcentia.com/api/v1/sql/bcb?key="+gc2APIKey
    return apiBaseURL

In [42]:
# Basic function to insert registration info pairs into TIR
def registerProperty(name,proptype,definition):
    # Check to see if property name already exists
    r = requests.get(getBaseURL()+"&q=SELECT gid FROM tir.propertyregistry WHERE name='"+name+"'").json()

    # Insert or update based on name query
    if len(r["features"]) == 0:
        q_property = getBaseURL()+"&q=INSERT INTO tir.propertyregistry (name,proptype,definition) VALUES ('"+name+"','"+proptype+"','"+definition+"')"
    else:
        q_property = getBaseURL()+"&q=UPDATE tir.propertyregistry SET proptype='"+proptype+"', definition='"+definition+"' WHERE name='"+name+"'"

    return requests.get(q_property).json()


This is an incomplete solution for this. I spent a few minutes trying to figure out a method using ipywidgets or web2py that would make this more of a lasting solution, but I ran out of time. I like the idea of using code something like this to drive the documentation of concepts being recorded in our data, but we should come up with something more robust.

What this next block does is work through all of the unique key names that have been recorded in the information fields for the TIR so far. Once it assembles that list, it iterates through and prompts for a property type and definition to go along with the name. We will probably refine the keys over time as I'm seeing a few things I would do differently with the concepts in retrospect. We will also want to add further definition to the keys, particularly some sort of URL reference to deeper level information about the concept. The information from this goes into a tir.propertyregistry table for use by applications.

In [43]:
r_registrationKeys = requests.get(getBaseURL()+"&q=SELECT array_agg(DISTINCT key) FROM (SELECT skeys(registration) AS key FROM tir.tir2) AS temp").json()
r_itisKeys = requests.get(getBaseURL()+"&q=SELECT array_agg(DISTINCT key) FROM (SELECT skeys(itis) AS key FROM tir.tir2) AS temp").json()
r_tessKeys = requests.get(getBaseURL()+"&q=SELECT array_agg(DISTINCT key) FROM (SELECT skeys(tess) AS key FROM tir.tir2) AS temp").json()
r_natureserveKeys = requests.get(getBaseURL()+"&q=SELECT array_agg(DISTINCT key) FROM (SELECT skeys(natureserve) AS key FROM tir.tir2) AS temp").json()

keyList = []
keyList.extend(r_registrationKeys["features"][0]["properties"]["array_agg"][1:-1].split(","))
keyList.extend(r_itisKeys["features"][0]["properties"]["array_agg"][1:-1].split(","))
keyList.extend(r_tessKeys["features"][0]["properties"]["array_agg"][1:-1].split(","))
keyList.extend(r_natureserveKeys["features"][0]["properties"]["array_agg"][1:-1].split(","))

for key in keyList:
    thisProperty = {}
    thisProperty["name"] = key
    thisProperty["proptype"] = ""
    thisProperty["definition"] = ""
    
    # Check to see if the property is already defined
    r_checkProperty = requests.get(getBaseURL()+"&q=SELECT * FROM tir.propertyregistry WHERE name='"+thisProperty["name"]+"'").json()
    if len(r_checkProperty["features"]) == 1:
        thisProperty["proptype"] = r_checkProperty["features"][0]["properties"]["proptype"]
        thisProperty["definition"] = r_checkProperty["features"][0]["properties"]["definition"]
    
    print (thisProperty["name"])
    thisProperty["proptype"] = input("Property Type: ")
    thisProperty["definition"] = input("Definition: ")
    print (registerProperty(thisProperty["name"],thisProperty["proptype"],thisProperty["definition"]))
    

DOI
Property Type: string
Definition: For GAP species registrations - the Digital Object Identifier assigned to the published habitat map
{'auth_check': {'session': None, 'success': True, 'auth_level': None}, 'affected_rows': 1, 'success': True, '_execution_time': 0.062}
EGTID
Property Type: integer
Definition: The numeric part of a NatureServe identifier for a species
{'auth_check': {'session': None, 'success': True, 'auth_level': None}, 'affected_rows': 1, 'success': True, '_execution_time': 0.061}
GAP_CommonName
Property Type: string
Definition: GAP species common name
{'auth_check': {'session': None, 'success': True, 'auth_level': None}, 'affected_rows': 1, 'success': True, '_execution_time': 0.059}
GAP_ScientificName
Property Type: GAP species scientific name
Definition: 
{'auth_check': {'session': None, 'success': True, 'auth_level': None}, 'affected_rows': 1, 'success': True, '_execution_time': 0.059}
GAP_SpeciesCode
Property Type: GAP species code
Definition: 
{'auth_check': {'